In [205]:
#Data-set includes Crude birth rate (per 1000 people per year) of 275 Countries for the years 1800-2015.
#Data-set taken from gapminder.org
import numpy as np
from bokeh.models import ColumnDataSource

import warnings
warnings.filterwarnings('ignore')

#Using Bokeh 0.12.5 and Python 3.6
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()

import xlrd
#Using XLRD for importing xlsx workbook
workbook=xlrd.open_workbook('Data/crude_birth_rate.xlsx')
sheet=workbook.sheet_by_index(0)

Loading BokehJS ...

In [206]:
#Reading country,year and birth rate data into NumPy arrays
cols=sheet.ncols
rows=sheet.nrows

data=[]
country_name=[]
year_name=[]

for x in range(rows-1):
    country_name.append(sheet.cell(x+1,0).value)
    data_row=[]
    for y in range(cols-1):
        if(sheet.cell(x+1,y+1).value==''):
            data_row.append(0)  #Filling missing values with 0s
            
        else:
            data_row.append(sheet.cell(x+1,y+1).value)
    data.append(data_row)

for x in range(cols-1):
    year_name.append(sheet.cell(0,x+1).value)
    

birth_rate=np.array(data,dtype=np.float32) # Float values 275*216
country=np.array(country_name) #Strings , 275
year=np.array(year_name,dtype=np.int32) #Taking years as ints, 216


In [212]:
# Fun Task 1 : Let's see how the average Crude birth rate over all countries looks like over the years

fig1 =figure(plot_width=800, plot_height=400, title="Average Global Crude Birth Rate")

fig1.line(year,birth_rate.mean(axis=0), line_width=3,color="green",legend="Global Average")
#fig1.circle(year,birth_rate.mean(axis=0), fill_color="white", size=3)


fig1.xaxis.axis_label = "Year"
fig1.yaxis.axis_label = "Crude Birth Rate"
fig1.axis.major_label_text_color = "#4B5C8A"
fig1.ygrid.grid_line_alpha = 0.4
fig1.ygrid.grid_line_dash = [5, 3]
fig1.xgrid.grid_line_alpha = 0.4
fig1.xgrid.grid_line_dash = [5, 3]

show(fig1)

#Well, it's decreasing, thankfully!

In [213]:
#Fun Task 2: Compare Crude Birth Rates of US, India and China with the global average. Hide/Unhide from legend
#Creating ColumnDataSource object
dictionary={}
mean=np.zeros((year.shape[0],))

for x in range(country.shape[0]):
    if(birth_rate[x,0]!=0): 
        #Removing Countries that have all 0 values, so that it doesn't affect average (elimination)
        dictionary[country[x]]=birth_rate[x,:]
        mean=mean+birth_rate[x,:]
dictionary['year']=year
dictionary['average']=mean/len(dictionary)

source = ColumnDataSource(data=dictionary)

#Plotting Crude birth rates of India, USA, China and Global Average (of countries with data)
fig2=figure(plot_width=800,plot_height=400, title="Comparing Crude Birth Rates of US,India,China and Global average")

fig2.line(x='year',y='India', line_width=2, color="#EC5546",legend='India ',source=source)
fig2.line(x='year',y='China', line_width=2, color="#71BBDD",legend="China ",source=source)
fig2.line(x='year',y='United States', line_width=2, color="#F3A256",legend="United States ",source=source)
fig2.line(x='year',y='average', line_width=2, color="green",legend="Global Average",line_dash=[4, 4],source=source)

#Axes and labels
fig2.xaxis.axis_label = "Year"
fig2.yaxis.axis_label = "Crude Birth Rate"
fig2.axis.major_label_text_color = "#4B5C8A"
fig2.ygrid.grid_line_alpha = 0.4
fig2.ygrid.grid_line_dash = [5, 3]
fig2.xgrid.grid_line_alpha = 0.4
fig2.xgrid.grid_line_dash = [5, 3]

fig2.legend.location = "top_right"
fig2.legend.click_policy="hide"
#Making legend clickable to hide/unhide glyphs

show(fig2)

#China has controlled their birth rates considerable whereas India is still behind

In [214]:
# Fun Task 3: Adding hovers on line graphs of India, China and Global average
#Hovers
from bokeh.models import HoverTool
hover_india = HoverTool(
        tooltips=[
            ("Country", "India"),
            ("(Year, CBR)", "(@year, @India)"),
        ],names=["India"]
    )
hover_china = HoverTool(
        tooltips=[
            ("Country", "China"),
            ("(Year, CBR)", "(@year, @China)"),
        ],names=["China"]
    )
hover_av = HoverTool(
        tooltips=[
            ("Country", "Global Average"),
            ("(Year, CBR)", "(@year, @average)"),
        ],names=["Average"]
    )
fig3=figure(plot_width=800,plot_height=300,tools=[hover_india,hover_china,hover_av],title="Hovering and Legend toggle Interaction")

fig3.line(x='year',y='India', line_width=2, color="#EC5546",legend='India ',source=source,name="India")
fig3.line(x='year',y='China', line_width=2, color="#71BBDD",legend="China ",source=source,name="China")
fig3.line(x='year',y='average', line_width=2, color="green",legend="World Average",line_dash=[4, 4],source=source,name="Average")

#Labels and Axes
fig3.xaxis.axis_label = "Year"
fig3.xaxis.axis_label_text_font='times'
fig3.yaxis.axis_label_text_font='times'
fig3.yaxis.axis_label = "Crude Birth Rate"
fig3.axis.major_label_text_color = "#4B5C8A"
fig3.ygrid.grid_line_alpha = 0.4
fig3.ygrid.grid_line_dash = [5, 3]
fig3.xgrid.grid_line_alpha = 0.4
fig3.xgrid.grid_line_dash = [5, 3]

fig3.legend.location = "top_right"
fig3.legend.click_policy="hide"
#Making legend clickable to hide/unhide glyphs

show(fig3)




In [215]:
#Fun Task 4: Choose country from drop-down to compare with global average. (Interactice)

from bokeh.models import Select,CustomJS
from bokeh.layouts import row

hover_av = HoverTool(
        tooltips=[
            ("Country", "Average"),
            ("(Year, CBR)", "(@year, @average)"),
        ],names=["Average"]
    )

country_chosen='Afghanistan'
#First country auto-selected

fig4=figure(plot_width=700,plot_height=400,tools=[hover_av], title="Choose country from drop-down to compare with global-average")
l3=fig4.line(x='year',y='average', line_width=2, color="green",legend="Global Average",line_dash=[4, 4],source=source,name="Average")
l4=fig4.line(x='year',y=country_chosen, line_width=2, color="#EC5546",legend="Country",source=source)

#Axes and Labels
fig4.xaxis.axis_label = "Year"
fig4.xaxis.axis_label_text_font='times'
fig4.yaxis.axis_label_text_font='times'
fig4.yaxis.axis_label = "Crude Birth Rate"
fig4.axis.major_label_text_color = "#4B5C8A"
fig4.ygrid.grid_line_alpha = 0.4
fig4.ygrid.grid_line_dash = [5, 3]
fig4.xgrid.grid_line_alpha = 0.4
fig4.xgrid.grid_line_dash = [5, 3]

fig4.legend.location = "top_right"
fig4.legend.click_policy="hide"
#Making legend clickable to hide/unhide glyphs

#Creating the single-select drop-down
menu=[]
for x in dictionary.keys():
    menu.append(tuple((x,x)))
    
drop_down=Select(title="Choose Country",value="Afghanistan", options=menu)
country_chosen=drop_down.value

#CustomJS callback for changing data interactively
update_curve = CustomJS(args=dict(source=source, drop_down=drop_down,l4=l4,fig4=fig4), code="""
    l4.glyph.y.field=drop_down.value;
    country_chosen=drop_down.value;
    source.trigger('change');
    //l4.visible=false;
""")

drop_down.js_on_change('value', update_curve)

show(row(fig4,drop_down))
